### Adjust Word Group Repeat With Youtube Link

In [11]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [12]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [13]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# file extention
file_ext = "Twogram_Threegram"

In [14]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [15]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 1/Result/3-Adjust Word Group Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

##### Link Count

In [ ]:
#### will be Talk Time 3

##### Link Repeat

In [16]:
path_folder_file = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Result/2-Find Word Group Youtube Link/Twogram Threegram Youtube Link Selected_Manuel.xlsx"
sheet = "Sheet1"  # Sheet1

In [17]:
df_word_group_time_loc = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,search_string,start_time,end_time,video_url,video_id,count
0,önemli olan,497,499,https://www.youtube.com/watch?v=NTG7NZgQals&t=...,NTG7NZgQals,7
1,önemli olan,1726,1728,https://www.youtube.com/watch?v=vsDoBhQJ1hY&t=...,vsDoBhQJ1hY,7
2,önemli olan,6246,6248,https://www.youtube.com/watch?v=m3Clq7uulPg&t=...,m3Clq7uulPg,7
3,önemli olan,406,408,https://www.youtube.com/watch?v=kzEvSIem_n4&t=...,kzEvSIem_n4,7
4,önemli olan,6184,6186,https://www.youtube.com/watch?v=JV6ZUdw_lXQ&t=...,JV6ZUdw_lXQ,7
...,...,...,...,...,...,...
583,bunun için,1005,1007,https://www.youtube.com/watch?v=oKfaa43c0hA&t=...,oKfaa43c0hA,4
584,bir daha,904,905,https://www.youtube.com/watch?v=pcAhoVDv2Gs&t=...,pcAhoVDv2Gs,4
585,bir daha,335,337,https://www.youtube.com/watch?v=jPFwUkNLd2M&t=...,jPFwUkNLd2M,4
586,bir daha,1813,1815,https://www.youtube.com/watch?v=BZ_kP3J6XcI&t=...,BZ_kP3J6XcI,4


In [18]:
search_list = df_word_group_time_loc.iloc[:,0].unique()
len(search_list)

97

In [19]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["search_string","start_time","end_time","sentence","video_id","video_url"])
df_link_default

,search_string,start_time,end_time,sentence,video_id,video_url
0,repeat,5,7,repeat_again,Q-8I-uMUMYA,https://www.youtube.com/watch?v=Q-8I-uMUMYA&t=0s


In [20]:
## All word group result convert to join result in one row
#df_result_repeat = pd.DataFrame()
#for word_group in search_list:
#    # for repeat 1
#    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
#    # for repeat 2
#    try:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
#    except:
#        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
#    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
#    # repeat block
#    for i in range(len(df_word_group_search_repeat1)):        
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)
#
#    try:
#        df_link_default_var = df_link_default
#        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
#        if word_time_diff_var < 4:
#            word_time_diff_var2 = 3
#        else:
#            word_time_diff_var2 = word_time_diff_var+1.0
#
#        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
#    except:
#        pass
#
#    for j in range(3):
#        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
#        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
#
#df_result_repeat.reset_index(drop=True, inplace=True)
#df_result_repeat_join = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"])
#
#df_result_repeat.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time2.xlsx", index=False)
#df_result_repeat_join.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Talk_Time_Join2.xlsx", index=False)

In [21]:
#df_result_repeat

In [22]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [23]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["search_string"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["search_string"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)


In [24]:
df_all_join_result

,video_id,start_time,end_time,search_string,count
0,"NTG7NZgQals,vsDoBhQJ1hY,m3Clq7uulPg,kzEvSIem_n...","497,1726,6246,406,6184,229,8797,8797,5,8797,5,...","499,1728,6248,408,6186,231,8800,8800,8,8800,8,...",önemli olan,7.0
1,"ncy57YkTtdU,cvsKxQ5nsZo,wH4L9NiHHIE,EC4gKcDZvu...","2514,62,6468,1395,353,1435,7343,2514,5,2514,5,...","2516,65,6470,1398,356,1437,7346,2516,8,2516,8,...",çok güzel değil,7.0
2,"zaHwaMoI314,tP3ou_3RGVg,iZjzaNo_vDk,LyvjoFQdsr...","286,6269,1228,131,240,2836,5735,1228,5,1228,5,...","288,6271,1230,133,242,2838,5737,1230,8,1230,8,...",çok güzel,7.0
3,"mpg2dM5DrGg,k1ni27c9C2k,oCsHNgql560,oCsHNgql56...","5691,535,5776,2315,209,558,3467,5776,5,5776,5,...","5692,537,5778,2317,212,559,3468,5778,8,5778,8,...",tamam mı,7.0
4,"GimA7_uhyWg,o0vdJUDd7hQ,od_ZPJBFYUA,J0CIPlClrb...","1635,356,6389,22,5280,2,32,356,5,356,5,356,5","1637,358,6391,23,5282,4,34,358,8,358,8,358,8",tam burada,7.0
...,...,...,...,...,...
92,"BEH6RWDGy7c,WyHI8VAPJzw,B2BcTrXcsVE,Qs1I7HxXYl...","4366,1617,998,2523,2523,5,2523,5,2523,5","4367,1619,1000,2525,2525,8,2525,8,2525,8",sen de,4.0
93,"tN1putCohn4,ud9s8aerIJE,M3pin36uaA4,x6aEnqpTUB...","4616,213,256,4099,213,5,213,5,213,5","4618,215,258,4101,215,8,215,8,215,8",izin ver,4.0
94,"ugLGoSZvuOc,ugLGoSZvuOc,MItD3hP_kqU,gIgjKheyn3...","6832,6848,476,2278,476,5,476,5,476,5","6834,6851,478,2280,478,8,478,8,478,8",burada neler oluyor,4.0
95,"s4MqASVecXc,agAUR7SzzH0,UV9gDdyggOw,oKfaa43c0h...","584,5631,57,1005,5631,5,5631,5,5631,5","586,5633,59,1007,5633,8,5633,8,5633,8",bunun için,4.0


#### Copy Move And Delete

In [26]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx")
output_file

['Turkish_200_Word_Youtube_Talk_Time_Join_Twogram_Threegram.xlsx']

In [27]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [28]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Temp

In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_excel("Twogram Threegram Youtube Link Selected_Manuel.xlsx", sheet_name="Sheet1")
df

,search_string,start_time,end_time,video_url,video_id
0,önemli olan,497,499,https://www.youtube.com/watch?v=NTG7NZgQals&t=...,NTG7NZgQals
1,önemli olan,1726,1728,https://www.youtube.com/watch?v=vsDoBhQJ1hY&t=...,vsDoBhQJ1hY
2,önemli olan,6246,6248,https://www.youtube.com/watch?v=m3Clq7uulPg&t=...,m3Clq7uulPg
3,önemli olan,406,408,https://www.youtube.com/watch?v=kzEvSIem_n4&t=...,kzEvSIem_n4
4,önemli olan,6184,6186,https://www.youtube.com/watch?v=JV6ZUdw_lXQ&t=...,JV6ZUdw_lXQ
...,...,...,...,...,...
583,sen de,2523,2525,https://www.youtube.com/watch?v=Qs1I7HxXYlg&t=...,Qs1I7HxXYlg
584,bir daha,904,905,https://www.youtube.com/watch?v=pcAhoVDv2Gs&t=...,pcAhoVDv2Gs
585,bir daha,335,337,https://www.youtube.com/watch?v=jPFwUkNLd2M&t=...,jPFwUkNLd2M
586,bir daha,1813,1815,https://www.youtube.com/watch?v=BZ_kP3J6XcI&t=...,BZ_kP3J6XcI


In [6]:
df_count = df.groupby(["search_string"])[["search_string"]].count()
df_count.rename(columns={"search_string":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,search_string,count
0,asla olmaz,7
1,tam burada,7
2,senin için,7
3,para için,7
4,orada mısın,7
...,...,...
92,bunun için,4
93,burada neler oluyor,4
94,bir daha,4
95,izin ver,4


In [7]:
df_count_merge = pd.merge(df, df_count, how="inner",on= "search_string")
df_count_merge.sort_values(by=["count","search_string"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,search_string,start_time,end_time,video_url,video_id,count
0,önemli olan,497,499,https://www.youtube.com/watch?v=NTG7NZgQals&t=...,NTG7NZgQals,7
1,önemli olan,1726,1728,https://www.youtube.com/watch?v=vsDoBhQJ1hY&t=...,vsDoBhQJ1hY,7
2,önemli olan,6246,6248,https://www.youtube.com/watch?v=m3Clq7uulPg&t=...,m3Clq7uulPg,7
3,önemli olan,406,408,https://www.youtube.com/watch?v=kzEvSIem_n4&t=...,kzEvSIem_n4,7
4,önemli olan,6184,6186,https://www.youtube.com/watch?v=JV6ZUdw_lXQ&t=...,JV6ZUdw_lXQ,7
...,...,...,...,...,...,...
583,bunun için,1005,1007,https://www.youtube.com/watch?v=oKfaa43c0hA&t=...,oKfaa43c0hA,4
584,bir daha,904,905,https://www.youtube.com/watch?v=pcAhoVDv2Gs&t=...,pcAhoVDv2Gs,4
585,bir daha,335,337,https://www.youtube.com/watch?v=jPFwUkNLd2M&t=...,jPFwUkNLd2M,4
586,bir daha,1813,1815,https://www.youtube.com/watch?v=BZ_kP3J6XcI&t=...,BZ_kP3J6XcI,4


In [8]:
df_count_merge["search_string"].nunique()

97

In [9]:
df_count_merge[df_count_merge["count"] == 1]

,search_string,start_time,end_time,video_url,video_id,count


In [10]:
df_count_merge.to_excel("Twogram Threegram Youtube Link Selected_Count.xlsx", index=False)

In [ ]:
df_word_group_time_loc_manuel = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Count_Manuel.xlsx")
df_word_group_time_loc_manuel.sort_values(by=["count","search_string"],inplace=True, ascending=False)
df_word_group_time_loc_manuel.reset_index(drop=True, inplace=True)
df_word_group_time_loc_manuel

In [ ]:
df_word_group_time_loc_manuel.to_excel("Twogram Threegram Youtube Link Selected_Manuel.xlsx", index=False)